In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['LD_LIBRARY_PATH'] = "$LD_LIBRARY_PATH:/usr/local/cuda-11.3/lib64/"
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork, Trajectory
#import torch
#import torch.nn as nn
#from torch.utils.data import DataLoader
#import pandas as pd
import networkx as nx
import osmnx as ox
from models.hrnr_original.utils import *
from models.hrnr_original.conf import beijing_hparams
# from models.hrnr_original.train import train_struct_cmt, train_fnc_cmt_rst
from models.hrnr_original.model import *
import torch
import torch.nn.functional as F
from torch import optim
from models.training.hrnr_data_generation import *
from models import HRNRModel

In [2]:
torch.cuda.set_device(0)

In [3]:
# # try on original
# hparams = dict_to_object(beijing_hparams)
# hparams.device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# adj, features, sp_label = load_gae_data(hparams) 

In [4]:
# print(len(np.unique(features[:, 2])))

In [5]:
# train_struct_cmt()

In [6]:
# train_fnc_cmt_rst()

In [3]:
city = "sf"
city_traj = "SF"

In [4]:
# load road network
network = RoadNetwork()
network.load(f"../../osm_data/{city}")
data = network.generate_road_segment_pyg_dataset(dataset=city)


In [9]:
len(network.gdf_edges["lanes"].unique())

30

In [5]:
trajectory = Trajectory(
    f"../../datasets/trajectories/{city_traj}/road_segment_map_final.csv", nrows=10000000
)

Pandas Apply:   0%|          | 0/249765 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/249765 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/249765 [00:00<?, ?it/s]

In [6]:
adjp, featuresp, sp_labelp, t_adjp = get_data(network, trajectory, remove_highway_label=True)

100%|██████████| 249765/249765 [01:07<00:00, 3689.80it/s]


In [7]:
# train structural region and functional region matrices
train_struct_cmt_custom(adjp, featuresp, sp_labelp, city=city)
train_fnc_cmt_rst_custom(adjp, featuresp, sp_labelp, t_adjp, city=city)


49.22737503051758
38.47576141357422
36.170379638671875
33.20859909057617
32.29621887207031
31.668087005615234
25.95848274230957
30.376855850219727
26.21424102783203
23.498126983642578
29.800891876220703
25.49786376953125
29.574195861816406
25.8492431640625
23.86736488342285
26.7635555267334
24.043630599975586
24.508726119995117
26.03940773010254
22.952178955078125
4.116616725921631
3.8864505290985107
3.8926877975463867
3.013582229614258
3.03020977973938
2.940955400466919
2.969712018966675
2.5990264415740967
2.227489471435547
2.8897335529327393
2.482081174850464
2.261179208755493
2.4742043018341064
2.6528003215789795
1.8561760187149048
2.2183432579040527
2.2133219242095947
2.497177839279175
1.7304935455322266
1.9051369428634644
1.8214532136917114
1.6278008222579956
2.024712085723877
2.060819387435913
1.4784035682678223
1.5917634963989258
1.9829844236373901
1.7166221141815186
1.7408286333084106
1.7275876998901367
1.7643285989761353
1.716855764389038
1.5653564929962158
1.591889500617981
1

In [8]:
# Build model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = HRNRModel(data, device=device, network=network, data_path="hrnr_data", remove_highway_label=True, city=city)

In [9]:
model.train()

Epoch: 50, avg_loss: 22.466595549583435
Epoch: 100, avg_loss: 11.953675824403764
Epoch: 150, avg_loss: 8.353836778004965
Epoch: 200, avg_loss: 6.54941038608551
Epoch: 250, avg_loss: 5.464239602565765
Epoch: 300, avg_loss: 4.755027819474538
Epoch: 350, avg_loss: 4.256304527350834
Epoch: 400, avg_loss: 3.8597656399011613
Epoch: 450, avg_loss: 3.5500486795107524
Epoch: 500, avg_loss: 3.301356628417969
Epoch: 550, avg_loss: 3.114794950918718
Epoch: 600, avg_loss: 2.94657015701135
Epoch: 650, avg_loss: 2.8014684988902165
Epoch: 700, avg_loss: 2.6763457027503423
Epoch: 750, avg_loss: 2.567779419104258
Epoch: 800, avg_loss: 2.4725009144842627
Epoch: 850, avg_loss: 2.3884937676261453
Epoch: 900, avg_loss: 2.3135558195908863
Epoch: 950, avg_loss: 2.24619830357401


In [10]:
model.save_model("../model_states/hrnr")

In [16]:
z = model.load_emb()

In [17]:
print(z.shape)

(11331, 128)
